In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize # Important pour l'étape 3

# -----------------------------------------------------------------
# ÉTAPE 0 : DONNÉES ET PRÉTRAITEMENT (Tokenisation)
# -----------------------------------------------------------------

# Corpus de texte d'exemple (le même que le vôtre)
corpus = [
    "Ceci est un exemple de phrase.",
    "Un autre exemple de phrase.",
    "Le modèle CBOW est utilisé pour prédire des mots en fonction du contexte.",
]

# Prétraitement : tokenisation et création du vocabulaire
# (C'est la fonction 'tokenize' que vous vouliez voir incluse)
def tokenize(corpus):
    tokens = [sentence.lower().split() for sentence in corpus]
    vocab = set([word for sentence in tokens for word in sentence])
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for word, idx in word2idx.items()}
    return tokens, word2idx, idx2word

# Exécution de la tokenisation
tokens, word2idx, idx2word = tokenize(corpus)
vocab_size = len(word2idx)

print(f"Taille du vocabulaire: {vocab_size}")
print(f"Dictionnaire (mot -> index): {word2idx}\n")


# -----------------------------------------------------------------
# ÉTAPE 1 : ADAPTATION VERS LE MODÈLE SKIP-GRAM
# -----------------------------------------------------------------

# 1.1. Préparer les données (mot -> contexte)
def create_skipgram_pairs(tokens, context_size=2):
    """
    Crée des paires (mot_cible, mot_contexte) pour le modèle Skip-gram.
    """
    skipgram_pairs = []
    for sentence in tokens:
        for i in range(len(sentence)):
            target_word = sentence[i]
            start = max(0, i - context_size)
            end = min(len(sentence), i + context_size + 1)
            
            for j in range(start, end):
                if i == j: # Ne pas appairer un mot avec lui-même
                    continue
                context_word = sentence[j]
                skipgram_pairs.append((target_word, context_word))
    return skipgram_pairs

# 1.2. Encoder les paires
def encode_skipgram_pairs(skipgram_pairs, word2idx):
    """
    Encode les paires (mot, mot) en (idx, idx).
    """
    encoded_pairs = []
    for target_word, context_word in skipgram_pairs:
        if target_word in word2idx and context_word in word2idx:
            target_idx = word2idx[target_word]
            context_idx = word2idx[context_word]
            encoded_pairs.append((target_idx, context_idx))
    return encoded_pairs

# 1.3. Définition du modèle Pytorch Skip-gram
class SkipGramModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGramModel, self).__init__()
        # La matrice d'embedding que l'on cherche à entraîner
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        # La couche de sortie
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, target_idx):
        embeds = self.embeddings(target_idx)
        out = self.linear(embeds)
        log_probs = torch.log_softmax(out, dim=1)
        return log_probs

# 1.4. Boucle d'entraînement
def train_skipgram(model, encoded_pairs, epochs=100, learning_rate=0.01):
    loss_function = nn.NLLLoss() # 'Negative Log Likelihood Loss'
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    print("--- Début de l'entraînement Skip-gram ---")
    for epoch in range(epochs):
        total_loss = 0
        for target_idx, context_idx in encoded_pairs:
            
            # Préparer les tenseurs (Entrée = Cible, Sortie = Contexte)
            target_var = torch.tensor([target_idx], dtype=torch.long)
            context_var = torch.tensor([context_idx], dtype=torch.long)

            # Forward pass
            model.zero_grad()
            log_probs = model(target_var)

            # Calcul de la perte (comparer la prédiction au mot de contexte)
            loss = loss_function(log_probs, context_var)

            # Backward pass et optimisation
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if epoch % 20 == 0 or epoch == epochs - 1:
            print(f'Époque {epoch:03d}, Perte (Loss): {total_loss:.4f}')
    print("--- Fin de l'entraînement ---")

# --- Exécution de l'Étape 1 ---
embedding_dim = 10 # 10 dimensions pour les vecteurs de mots
context_size = 2

skipgram_pairs = create_skipgram_pairs(tokens, context_size)
encoded_pairs = encode_skipgram_pairs(skipgram_pairs, word2idx)

model = SkipGramModel(vocab_size, embedding_dim)
train_skipgram(model, encoded_pairs, epochs=100)

# Extraire la matrice d'embedding finale (le résultat de l'étape 1)
embedding_matrix = model.embeddings.weight.detach().numpy()
print(f"\nMatrice d'embedding (Taille: {embedding_matrix.shape}) extraite.")


# -----------------------------------------------------------------
# ÉTAPE 2 : VISUALISATION (ACP / PCA)
# -----------------------------------------------------------------

print("\n--- Début de l'Étape 2 : Visualisation ACP (PCA) ---")

# Appliquer l'ACP pour réduire de 'embedding_dim' à 2 dimensions
pca = PCA(n_components=2)
embedding_2d = pca.fit_transform(embedding_matrix)

# Créer la visualisation
plt.figure(figsize=(14, 10))
for i in range(vocab_size):
    x, y = embedding_2d[i]
    plt.scatter(x, y, color='blue')
    plt.annotate(idx2word[i], # Texte (le mot)
                 (x, y),       # Position
                 ha='center',  # Alignement horizontal
                 xytext=(0, 5),# Décalage du texte
                 textcoords='offset points')
    
plt.title("Visualisation 2D des Embeddings de Mots (via ACP/PCA)")
plt.xlabel("Composante Principale 1")
plt.ylabel("Composante Principale 2")
plt.grid(True)
plt.show() # Affiche le graphique

print("Visualisation ACP terminée.")


# -----------------------------------------------------------------
# ÉTAPE 3 : CLUSTERING (K-MEANS)
# -----------------------------------------------------------------

print("\n--- Début de l'Étape 3 : Clustering K-Means ---")

# !!! ATTENTION (votre consigne) !!!
# Nous devons utiliser la Similarité Cosinus.
# L'astuce consiste à normaliser les vecteurs (norme L2).
# KMeans (qui utilise la distance euclidienne) sur des données normalisées
# est mathématiquement équivalent à un clustering basé sur la similarité cosinus.
embedding_matrix_normalized = normalize(embedding_matrix, norm='l2', axis=1)
print("Matrice d'embedding normalisée (pour similarité cosinus).")

# Choisir 'k' (nombre de groupes)
# En se basant sur la visualisation ACP, on peut essayer k=3 ou k=4
k = 3 
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10) # n_init=10 évite un avertissement
clusters = kmeans.fit_predict(embedding_matrix_normalized)

# 4. Interpréter les groupes
grouped_words = {i: [] for i in range(k)}

# 'idx2word' est {0: 'mot1', 1: 'mot2', ...}
# 'clusters' est [0, 2, 1, 0, ...] (le groupe de chaque mot)
for word_idx, cluster_id in enumerate(clusters):
    word = idx2word[word_idx]
    grouped_words[cluster_id].append(word)

# Afficher les résultats
print(f"\n--- Interprétation des {k} groupes (K-Means) ---")
for i in range(k):
    print(f"\n--- GROUPE {i+1} ---")
    # Affiche les mots du groupe
    print(", ".join(grouped_words[i]))

print("\n(L'interprétation sera meilleure avec un corpus plus grand)")
print("--- Fin du devoir ---")